In [61]:
import os
os.environ['OMP_NUM_THREADS'] = '2'
os.environ['export OPENBLAS_NUM_THREADS']='2'

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from utilities3 import *

import operator
from functools import reduce
from functools import partial

from timeit import default_timer

from Adam import Adam

import pickle

torch.manual_seed(0)
np.random.seed(0)

In [62]:
################################################################
# 3d fourier layers
################################################################

class SpectralConv3d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1, modes2, modes3):
        super(SpectralConv3d, self).__init__()

        """
        3D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1 #Number of Fourier modes to multiply, at most floor(N/2) + 1
        self.modes2 = modes2
        self.modes3 = modes3

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))
        self.weights2 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))
        self.weights3 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))
        self.weights4 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, self.modes2, self.modes3, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul3d(self, input, weights):
        # (batch, in_channel, x,y,t ), (in_channel, out_channel, x,y,t) -> (batch, out_channel, x,y,t)
        return torch.einsum("bixyz,ioxyz->boxyz", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfftn(x, dim=[-3,-2,-1])
        
        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-3), x.size(-2), x.size(-1)//2 + 1, dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.modes1, :self.modes2, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, :self.modes1, :self.modes2, :self.modes3], self.weights1)
        out_ft[:, :, -self.modes1:, :self.modes2, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, -self.modes1:, :self.modes2, :self.modes3], self.weights2)
        out_ft[:, :, :self.modes1, -self.modes2:, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, :self.modes1, -self.modes2:, :self.modes3], self.weights3)
        out_ft[:, :, -self.modes1:, -self.modes2:, :self.modes3] = \
            self.compl_mul3d(x_ft[:, :, -self.modes1:, -self.modes2:, :self.modes3], self.weights4)

        #Return to physical space
        x = torch.fft.irfftn(out_ft, s=(x.size(-3), x.size(-2), x.size(-1)))
        return x

class FNO3d(nn.Module):
    def __init__(self, modes1, modes2, modes3, width):
        super(FNO3d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the first 10 timesteps + 3 locations (u(1, x, y), ..., u(10, x, y),  x, y, t). It's a constant function in time, except for the last index.
        input shape: (batchsize, x=64, y=64, t=40, c=13)
        output: the solution of the next 40 timesteps
        output shape: (batchsize, x=64, y=64, t=40, c=1)
        """

        self.modes1 = modes1
        self.modes2 = modes2
        self.modes3 = modes3
        self.width = width
        self.padding = 6 # pad the domain if input is non-periodic
        self.fc0 = nn.Linear(13, self.width)
        # self.fc0 = nn.Linear(8, self.width)
        # input channel is 12: the solution of the first 10 timesteps + 3 locations (u(1, x, y), ..., u(10, x, y),  x, y, t)

        self.conv0 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.conv1 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.conv2 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.conv3 = SpectralConv3d(self.width, self.width, self.modes1, self.modes2, self.modes3)
        self.w0 = nn.Conv3d(self.width, self.width, 1)
        self.w1 = nn.Conv3d(self.width, self.width, 1)
        self.w2 = nn.Conv3d(self.width, self.width, 1)
        self.w3 = nn.Conv3d(self.width, self.width, 1)
        self.bn0 = torch.nn.BatchNorm3d(self.width)
        self.bn1 = torch.nn.BatchNorm3d(self.width)
        self.bn2 = torch.nn.BatchNorm3d(self.width)
        self.bn3 = torch.nn.BatchNorm3d(self.width)

        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 4, 1, 2, 3)
        x = F.pad(x, [0,self.padding]) # pad the domain if input is non-periodic

        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv1(x)
        x2 = self.w1(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv2(x)
        x2 = self.w2(x)
        x = x1 + x2
        x = F.gelu(x)

        x1 = self.conv3(x)
        x2 = self.w3(x)
        x = x1 + x2

        x = x[..., :-self.padding]
        x = x.permute(0, 2, 3, 4, 1) # pad the domain if input is non-periodic
        x = self.fc1(x)
        x = F.gelu(x)
        x = self.fc2(x)
        return x

    def get_grid(self, shape, device):
        batchsize, size_x, size_y, size_z = shape[0], shape[1], shape[2], shape[3]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1, 1).repeat([batchsize, 1, size_y, size_z, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1, 1).repeat([batchsize, size_x, 1, size_z, 1])
        gridz = torch.tensor(np.linspace(0, 1, size_z), dtype=torch.float)
        gridz = gridz.reshape(1, 1, 1, size_z, 1).repeat([batchsize, size_x, size_y, 1, 1])
        return torch.cat((gridx, gridy, gridz), dim=-1).to(device)

In [63]:
################################################################
# configs
################################################################
DATA_PATH = 'Solutions/acc_rate_initial_constant.npy'

# currently data are 370 samples
ntrain = 300
ntest = 70

modes = 8
width = 20

batch_size = 10

epochs = 100 #500
learning_rate = 0.001
scheduler_step = 100
scheduler_gamma = 0.5

print(epochs, learning_rate, scheduler_step, scheduler_gamma)

path = 'acc_rate3d_constant_ep500_Tin1'
path_model = 'model/'+path
path_train_err = 'results/'+path+'train.txt'
path_test_err = 'results/'+path+'test.txt'
path_image = 'image/'+path

runtime = np.zeros(2, )
t1 = default_timer()

S1 = 65
S2 = 97
T_in = 10
T = 10
step = 1

100 0.001 100 0.5


In [64]:
################################################################
# load data
################################################################
data_gen = np.load(DATA_PATH)

train_a = torch.tensor(data_gen[:ntrain,:,:,:T_in], dtype=torch.float)
train_u = torch.tensor(data_gen[:ntrain,:,:,T_in:T+T_in], dtype=torch.float)

test_a = torch.tensor(data_gen[-ntest:,:,:,:T_in], dtype=torch.float)
test_u = torch.tensor(data_gen[-ntest:,:,:,T_in:T+T_in], dtype=torch.float)

print(train_u.shape)
print(test_a.shape)
assert (S1 == train_u.shape[-3])
assert (S2 == train_u.shape[-2])
assert (T == train_u.shape[-1])


a_normalizer = UnitGaussianNormalizer(train_a)
with open('a_normalizer_constant.pkl', 'wb') as f:
    pickle.dump(a_normalizer, f)

train_a = a_normalizer.encode(train_a)
test_a = a_normalizer.encode(test_a)

y_normalizer = UnitGaussianNormalizer(train_u)
with open('y_normalizer_constant.pkl', 'wb') as f:
    pickle.dump(y_normalizer, f)
train_u = y_normalizer.encode(train_u)

train_a =  train_a.reshape(ntrain,S1,S2,1,T_in).repeat([1,1,1,T,1])
test_a = test_a.reshape(ntest,S1,S2,1,T_in).repeat([1,1,1,T,1])

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_a, train_u), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

t2 = default_timer()

print('preprocessing finished, time used:', t2-t1)
device = torch.device('cuda')

torch.Size([300, 65, 97, 10])
torch.Size([70, 65, 97, 10])
preprocessing finished, time used: 0.5126456599973608


In [65]:
################################################################
# training and evaluation
################################################################
model = FNO3d(modes, modes, modes, width).cuda()

print(count_params(model))
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)


myloss = LpLoss(size_average=False)
y_normalizer.cuda()
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        optimizer.zero_grad()
        out = model(x).view(batch_size, S1, S2, T)
        
        mse = F.mse_loss(out, y, reduction='mean')
        # mse.backward()

        y = y_normalizer.decode(y)
        out = y_normalizer.decode(out)
        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward()

        optimizer.step()
        train_mse += mse.item()
        train_l2 += l2.item()

    scheduler.step()

    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x).view(batch_size, S1, S2, T)
            out = y_normalizer.decode(out)
            test_l2 += myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()

    train_mse /= len(train_loader)
    train_l2 /= ntrain
    test_l2 /= ntest

    t2 = default_timer()
    print(ep, t2-t1, train_mse, train_l2, test_l2)
# torch.save(model, path_model)

6558537
0 2.390055518000736 1.0105550050735475 0.2824845862388611 0.41631597450801305
1 2.3734334479959216 0.37999996592601143 0.1226729999979337 0.0609959534236363
2 2.3734270589920925 0.05032013282179833 0.03531725501020749 0.013403576399598802
3 2.3777048680058215 0.04417330486079057 0.02195672002931436 0.010307077052337783
4 2.3781967779941624 0.03821157334993283 0.008019686502714951 0.010417680814862251
5 2.3784668689913815 0.038026850391179326 0.005874524340033532 0.008810733790908541
6 2.377216007997049 0.03893498343725999 0.01022142868489027 0.007304637293730464
7 2.3790640279912623 0.038784720748662946 0.008429095049699147 0.010897845242704663
8 2.379169837993686 0.03821921348571777 0.005127705441166957 0.006698771406497274
9 2.3789898690010887 0.0383935176456968 0.005930116275946299 0.006689929483192307
10 2.382401088005281 0.039071280198792614 0.008288837286333244 0.01872959498848234
11 2.3880387070094002 0.03882716000080109 0.0069568954532345136 0.014267427261386599
12 2.38

In [66]:
pred = torch.zeros(test_u.shape)

index = 0
# model = torch.load("model/acc_rate3d_constant_ep100_Tin10")
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=1, shuffle=False)
first_output = 0

with torch.no_grad():
    for x, y in test_loader:
        test_l2 = 0
        if index == 0:
            first_output = y.clone()
        x, y = x.cuda(), y.cuda()
        out = model(x)
        out = y_normalizer.decode(out[:,:,:,0])
        pred[index] = out

        test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
        print(index, test_l2)

# for i in range(18):
#     cp = plt.imshow(abs(pred[0,:,:,i]-first_output[0,:,:,i]))
#     plt.colorbar(cp)
#     plt.show()
#     print(i)
    # cp = plt.imshow(first_output[0,:,:,i])
    # plt.colorbar(cp)
    # plt.show()

0 0.018128719180822372
0 0.018193913623690605
0 0.018262118101119995
0 0.018333857879042625
0 0.018414774909615517
0 0.018504535779356956
0 0.018602078780531883
0 0.01870839111506939
0 0.01882297918200493
0 0.018945304676890373
0 0.01907384768128395
0 0.019194385036826134
0 0.019456975162029266
0 0.01974499225616455
0 0.02003481797873974
0 0.02032717876136303
0 0.020612208172678947
0 0.020895246416330338
0 0.021173207089304924
0 0.02143574133515358
0 0.021675173193216324
0 0.02190515026450157
0 0.022128963842988014
0 0.0223232414573431
0 0.022487416863441467
0 0.02264459803700447
0 0.022777611389756203
0 0.022883661091327667
0 0.02299516461789608
0 0.023112352937459946
0 0.02323441207408905
0 0.023354798555374146
0 0.023476962000131607
0 0.02360466495156288
0 0.023737527430057526
0 0.023871874436736107
0 0.023997390642762184
0 0.024119224399328232
0 0.024244220927357674
0 0.024373725056648254
0 0.024640394374728203
0 0.0251249298453331
0 0.025620348751544952
0 0.02610672637820244
0 0.0